# 챗봇-기본: 허깅페이스 TGI 서버 활용

## 0. 라이브러리 설치

In [1]:
!pip install pypdf # pypdf
!pip install spacy # spaCy
!pip install langchain # LangChain Official
!pip install langchain-core # Langchain Core
!pip install langchain-openai # LangChain Open-AI
!pip install langchain-community # LangChain Community
!pip install langchain-huggingface # LangChain Huggingface
!pip install sentence-transformers # Sentence Transformers

## 1. 패키지 로드

In [140]:
from typing import List, Union
import os
import faiss
import getpass
import numpy as np
from operator import itemgetter
from pydantic import BaseModel, Field
from langchain_openai import OpenAI
from langchain_huggingface import (
    HuggingFaceEndpoint,
    HuggingFaceEmbeddings
)
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import SpacyTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import PromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

## 2. 모델 로드

### 1) Split Model

In [3]:
# spaCy 모델 다운로드
!python -m spacy download ko_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 MB 26.6 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_lg')


In [4]:
# spaCy Text Splitter Model 로드 옵션
spacy_pipeline = "ko_core_news_lg" # spaCy 모델명
spacy_chunk_size = 300 # 최대 청크 크기
spacy_chunk_overlap = 50 # 청크 중복 허용 크기

# spaCy Text Splitter Model 로드
text_splitter_model = SpacyTextSplitter(
    pipeline=spacy_pipeline,
    chunk_size=spacy_chunk_size,
    chunk_overlap=spacy_chunk_overlap
)

### 2) Embedding Model

In [5]:
# Embedding Model 로드 옵션
emb_model_name = "intfloat/multilingual-e5-large-instruct" # HuggingFace 모델 Repository
emb_model_kwargs = {'device': 'cuda'} # 연산 장치 선택 ("cpu" or "cuda")
emb_encode_kwargs = {'normalize_embeddings': False} # 정규화 여부 선택
emb_model_cache_folder = "../../models/" # 모델 가중치 저장 폴더 경로

# Embedding Model 로드
embedding_model = HuggingFaceEmbeddings(
    model_name=emb_model_name,
    model_kwargs=emb_model_kwargs,
    encode_kwargs=emb_encode_kwargs,
    cache_folder=emb_model_cache_folder
)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3) LLM(Large Language Model)
*OpenAI LLM 또는 HuggingFace LLM 선택*

#### (1) OpenAI 모델

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [65]:
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
)

#### (2) HuggingFace 모델

## 3. 벡터DB

### 1) 벡터DB - 참고자료

In [149]:
folder_path_1 = None # 기존 저장한 DB가 있는 경우 "폴더 경로" or "None"
index_name_1 = None # 기존 저장한 DB의 인덱스 이름

# 구축한 DB가 없는 경우
if folder_path_1 is None:
    # 1. 데이터 로드
    pdf_loader = PyPDFLoader("../../data/Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf")
    pdf_docs = pdf_loader.load()

    # 2. 데이터 청킹
    chunks = text_splitter_model.split_documents(pdf_docs)

    # 3. 데이터 저장
    relevant_info_db = FAISS.from_documents(documents=chunks, embedding=embedding_model)
else: # 구축한 DB가 있는 경우
    # 1. DB 로드
    relevant_info_db = FAISS.load_local(
        folder_path=folder_path_1,
        index_name=index_name_1,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True,
    )

### 2) 벡터DB - 참고대화

In [150]:
folder_path_2 = None # 기존 저장한 DB가 있는 경우 "폴더 경로" or "None"
index_name_2 = None # 기존 저장한 DB의 인덱스 이름

# 구축한 DB가 없는 경우
if folder_path_2 is None:
    dimension_size = len(embedding_model.embed_query("hello world"))
    
    relevant_conv_db = FAISS(
        embedding_function=embedding_model,
        index=faiss.IndexFlatL2(dimension_size),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
else: # 구축한 DB가 있는 경우
    # 1. DB 로드
    relevant_conv_db = FAISS.load_local(
        folder_path=folder_path_2,
        index_name=index_name_2,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True,
    )

## 4. 메모리

### 1) Previous Conv Memory

In [193]:
prev_conv_memory = []

def get_previous_message(message: str) -> str:
    """이전 대화내용을 반환하는 함수"""
    return "\n".join(prev_conv_memory[:2]) if len(prev_conv_memory) > 1 else ""

### 2) Relevant Conv Memory

In [194]:
relevant_conv_k = 1 # 검색결과 중 가져올 개수 설정

def get_relevant_message(message: str) -> str:
    """사용자의 질문과 관련된 내용을 벡터DB에서 검색하여 반환하는 함수"""
    # 검색
    results = relevant_conv_db.similarity_search(query=message, k=relevant_conv_k)

    # 포맷팅
    content_list = []
    for result in results:
        content = result.page_content

        content_list.append(content)

    return "\n\n".join(content_list)

### 3) Relevant Info Memory

In [195]:
relevant_info_k = 1 # 검색결과 중 가져올 개수 설정

def get_relevant_information(message: str) -> str:
    """사용자의 질문과 관련된 정보를 벡터DB에서 검색하여 반환하는 함수"""
    # 검색
    results = relevant_info_db.similarity_search(query=message, k=relevant_info_k)

    # 포맷팅
    content_list = []
    for result in results:
        title = os.path.splitext(os.path.basename(result.metadata["source"]))[0]
        content = result.page_content
        page_num = result.metadata["page_label"]

        content_list.append(f"# {title} - p.{page_num}\n## {content}")
    
    return "\n\n".join(content_list)

## 5. 챗봇

### 1) Prompt

In [196]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """당신은 사용자의 질문에 대해서 정확하게 답변하는 유능한 챗봇입니다. 아래에 관련된 정보가 있다면 참고하여 답변하세요.

    [참고대화]
    {relevant_conv}
    
    [참고정보]
    {relevant_info}

    이전 대화:
    {prev_conv}

    Human: {query}
    AI:"""
)

### 2) Chain

In [197]:
# 메모리를 LCEL에 호환되는 형식으로 변경(Runnable 객체)
previous_conv = RunnableLambda(lambda x: get_previous_message(x["query"]))
relevant_conv = RunnableLambda(lambda x: get_relevant_message(x["query"]))
relevant_info = RunnableLambda(lambda x: get_relevant_information(x["query"]))

# 입력 쿼리에 대한 Runnable 객체 정의(그대로 통과하도록 설정)
query_passthrough = RunnableLambda(lambda x: x["query"])

# Chain에 대한 전체 입력을 병합하는 Runnable 객체 정의
full_input = RunnableMap(
    {
        "prev_conv": previous_conv,
        "relevant_conv": relevant_conv,
        "relevant_info": relevant_info,
        "query": query_passthrough,
    }
)

# chain 정의
chain = full_input | prompt_template | llm | StrOutputParser()

## 6. 테스트

### 1) LLM 챗봇 함수 정의

In [198]:
def chat_with_memory(query: str, is_stream: bool=False, is_debug: bool=False) -> str:
    """LLM 챗봇 함수"""
    # 사용자 입력 포맷팅
    user_input = {"query" : query}

    # Debug 모드
    if is_debug:
        config = {"callbacks": [ConsoleCallbackHandler()]}
    else:
        config = {}
    
    # Stream 모드
    if is_stream:
        response_chunks = []
        for chunk in chain.stream(user_input, config=config):
            print(chunk, end="", flush=True)
            response_chunks.append(chunk)
        full_response = "".join(response_chunks)
    else:
        full_response = chain.invoke(user_input, config=config)
        print(full_response)
    
    # prev_conv 메모리 업데이트
    prev_conv_memory.append(f"Human: {query}")
    prev_conv_memory.append(f"AI: {full_response}")

    # relevant_conv 메모리 업데이트
    relevant_conv_db.add_documents([Document(page_content=f"user: {query}\nai: {full_response}")])

### 2) 채팅

In [199]:
is_stream = True # 스트리밍 출력 설정
is_debug = False # 디버그 설정

In [200]:
print("🤖 챗봇과 대화를 시작합니다. 종료하려면 '종료'를 입력하세요.\n")
while True:
    # 질문 입력
    user_query = input("👤 사용자: ")

    # 종료
    if user_query.strip().lower() in ["종료"]:
        print("\n✅ 채팅을 종료합니다.")
        break

    # 챗봇 출력
    print("🤖 챗봇:", end=" ", flush=True)
    chat_with_memory(user_query, is_stream=is_stream, is_debug=is_debug)
    print("\n")

🤖 챗봇과 대화를 시작합니다. 종료하려면 '종료'를 입력하세요.



👤 사용자:  안녕하세요


🤖 챗봇: [chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<prev_conv,relevant_conv,relevant_info,query>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<prev_conv,relevant_conv,relevant_info,query> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<prev_conv,relevant_conv,relevant_info,query> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<prev_conv,relevant_conv,relevant_info,query> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<prev_conv,relevant_conv,relevant_info,query> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:Runna

👤 사용자:  종료



✅ 채팅을 종료합니다.


## 7. 벡터DB 저장

### 1) 참고자료 DB

In [126]:
relevant_info_db.save_local("../../db/faiss/relevant_info")

### 2) 참고대화 DB

In [125]:
relevant_conv_db.save_local("../../db/faiss/relevant_conv")